In [13]:
from os import listdir
from pickle import dump
import keras
# from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import os
from PIL import Image
import pandas as pd

import tqdm
import matplotlib.pyplot as plt
import random 
import tensorflow
import numpy.core.multiarray
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D


from keras.models import load_model

import sklearn
import subprocess
import time
# import PIL
c_folder = None
import dlib


In [14]:
lista_images = []
lista_labels = []
main_folder = r'C:\\Users\\vinic\\Desktop\\Modelagem-Antispoofing\\Modelo-Spoof-Printed\\treino'
pastas = os.listdir(main_folder)
for i  in range(len(pastas)):
    image  = load_img('C:\\Users\\vinic\\Desktop\\Modelagem-Antispoofing\\Modelo-Spoof-Printed\\treino\\'+str(pastas[i]),target_size=(224,224))
    lista_images.append(image)
    lista_labels.append(pastas[i])

In [15]:
from keras.applications import  resnet50
# model_resnet = resnet50.ResNet50(weights='imagenet')
model_resnet = resnet50.ResNet50(weights='imagenet')
#Modify model to remove the last layer
model_resnet.layers.pop()
model_resnet = Model(inputs=model_resnet.inputs,outputs=model_resnet.layers[-1].output)
print(model_resnet.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

activation_108 (Activation)     (None, 56, 56, 256)  0           add_35[0][0]                     
__________________________________________________________________________________________________
res3a_branch2a (Conv2D)         (None, 28, 28, 128)  32896       activation_108[0][0]             
__________________________________________________________________________________________________
bn3a_branch2a (BatchNormalizati (None, 28, 28, 128)  512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_109 (Activation)     (None, 28, 28, 128)  0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, 28, 28, 128)  147584      activation_109[0][0]             
__________________________________________________________________________________________________
bn3a_branc

__________________________________________________________________________________________________
bn3d_branch2c (BatchNormalizati (None, 28, 28, 512)  2048        res3d_branch2c[0][0]             
__________________________________________________________________________________________________
add_39 (Add)                    (None, 28, 28, 512)  0           bn3d_branch2c[0][0]              
                                                                 activation_117[0][0]             
__________________________________________________________________________________________________
activation_120 (Activation)     (None, 28, 28, 512)  0           add_39[0][0]                     
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, 14, 14, 256)  131328      activation_120[0][0]             
__________________________________________________________________________________________________
bn4a_branc

__________________________________________________________________________________________________
bn4d_branch2b (BatchNormalizati (None, 14, 14, 256)  1024        res4d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_131 (Activation)     (None, 14, 14, 256)  0           bn4d_branch2b[0][0]              
__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, 14, 14, 1024) 263168      activation_131[0][0]             
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_43 (Add)                    (None, 14, 14, 1024) 0           bn4d_branch2c[0][0]              
          

__________________________________________________________________________________________________
bn5b_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2a[0][0]             
__________________________________________________________________________________________________
activation_142 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2a[0][0]              
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359808     activation_142[0][0]             
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_143 (Activation)     (None, 7, 7, 512)    0           bn5b_branch2b[0][0]              
__________

In [16]:
lista_vetores_extraidos = [] 

for i in range(len(lista_images)):
    if(i%100==0):
        print(i)
    image = lista_images[i]
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    result = model_resnet.predict(image, verbose=0)[0]
    lista_vetores_extraidos.append(result)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [17]:
data = pd.DataFrame(lista_vetores_extraidos)
data.to_csv('lista_vetores_extraidos.csv')

In [18]:
data_full = pd.read_csv('lista_vetores_extraidos.csv') 

In [19]:
data_full.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0.000000,1.547326,0.047399,0.379688,0.349150,0.000000,0.821415,0.322773,0.033930,...,0.418763,0.347225,1.348159,1.999201,0.124287,0.089141,0.113760,0.044561,0.000000,0.221900
1,1,0.130637,0.676029,0.034060,3.512516,0.000000,0.000000,0.021712,0.045708,0.051035,...,0.000000,0.065273,0.350916,3.659316,0.038164,0.000000,0.002582,1.274637,0.000000,0.020711
2,2,0.086924,0.077958,0.019745,0.840674,0.237669,0.000000,1.171796,5.482133,0.000000,...,0.263594,0.000000,0.094521,0.498497,0.135649,0.036464,0.028075,0.751359,0.008199,0.295892
3,3,0.149071,0.209564,0.052960,5.727815,0.251091,0.139893,0.388269,0.059836,0.059026,...,0.177728,1.776933,0.281822,0.321183,0.047752,0.132728,0.000000,1.832905,0.187644,0.352156
4,4,0.433165,0.579551,0.033402,0.213514,0.377541,0.069415,0.165528,0.618216,0.135642,...,0.011909,0.223022,1.466870,4.102743,0.147598,0.022354,0.809533,0.000000,0.128264,0.299370


In [20]:
y = []
for i in range(len(lista_labels)):
    labels=pd.DataFrame(lista_labels)
    a = labels.iloc[i:i+1]
    b = a.values[0][0]
    c = b.split('(')
    y.append(c)

In [21]:
Y = pd.DataFrame(y)
y_all = Y.iloc[:,0:2]

In [22]:
y_all

,0,1
0,real,1).jpg
1,real,1).PNG
2,real,10).jpg
3,real,10).PNG
4,real,100).jpg
5,real,100).PNG
6,real,101).jpg
7,real,101).PNG
8,real,102).jpg
9,real,102).PNG


In [23]:
y_all.columns = ['y','formato']

In [24]:
y_all.to_csv('y.csv')

In [25]:
data_full =  data_full.iloc[:,1:]

In [26]:
data_full.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,1.547326,0.047399,0.379688,0.349150,0.000000,0.821415,0.322773,0.033930,0.310530,...,0.418763,0.347225,1.348159,1.999201,0.124287,0.089141,0.113760,0.044561,0.000000,0.221900
1,0.130637,0.676029,0.034060,3.512516,0.000000,0.000000,0.021712,0.045708,0.051035,0.258569,...,0.000000,0.065273,0.350916,3.659316,0.038164,0.000000,0.002582,1.274637,0.000000,0.020711
2,0.086924,0.077958,0.019745,0.840674,0.237669,0.000000,1.171796,5.482133,0.000000,0.199429,...,0.263594,0.000000,0.094521,0.498497,0.135649,0.036464,0.028075,0.751359,0.008199,0.295892
3,0.149071,0.209564,0.052960,5.727815,0.251091,0.139893,0.388269,0.059836,0.059026,0.426400,...,0.177728,1.776933,0.281822,0.321183,0.047752,0.132728,0.000000,1.832905,0.187644,0.352156
4,0.433165,0.579551,0.033402,0.213514,0.377541,0.069415,0.165528,0.618216,0.135642,1.276618,...,0.011909,0.223022,1.466870,4.102743,0.147598,0.022354,0.809533,0.000000,0.128264,0.299370


In [27]:
len(data_full)

1352

In [28]:
# y_all.iloc[i:i+1,0:1].values[0][0]

text = y_all.iloc[0:1,0:1].values[0][0]
text

'real '

In [29]:
text = y_all.iloc[1:2,0:1].values[0][0]
text

'real '

In [30]:
y = []
a = 1
b = 0
for i in range(len(data_full)):
#     print(y_all.iloc[i:i+1,0:1].values[0][0])
    if (y_all.iloc[i:i+1,0:1].values[0][0] == 'spoof '):
        y.append(a)
    else:
        y.append(b)


In [31]:
Y = pd.DataFrame(y)
Y.columns = ['Y']

In [32]:
Y.sum()

Y    731
dtype: int64

# Modelagem

In [33]:
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('notebook')

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics  import auc, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

### Split Dataset - treino / teste

In [34]:
X_all = data_full
y_all = Y.Y

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.25 ,random_state = 79831)

# id_train= X_train.iloc[:,0:2] 
# id_test = X_test.iloc[:,0:2]

#X_train = X_train.iloc[:,2:]
#X_test = X_test.iloc[:,2:]

print('Numero de observaçoes do treino:', len(X_train))
print('Numero de observaçoes da teste:',len(X_test))

Numero de observaçoes do treino: 1014
Numero de observaçoes da teste: 338


In [35]:
# Standardizing the features
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

std = StandardScaler()
X_train_2 = std.fit_transform(X_train)
X_test_2 = std.transform(X_test)

In [36]:
pca = PCA(n_components=800)
X_train_3 = pca.fit_transform(X_train_2)
X_test_3 = pca.transform(X_test_2)

In [ ]:
pca.explained_variance_ratio_.sum()

0.9938729609917323

In [ ]:
import time
from sklearn.metrics  import average_precision_score, make_scorer, roc_curve,f1_score, precision_score, recall_score, fbeta_score, auc, roc_auc_score, accuracy_score, confusion_matrix, classification_report,precision_recall_curve

### ------------------------------------------------------------------------------------------------------------------------------------------------------------

## RandomForestClassifier

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state = 665)

params = {'n_estimators' : [100,200,300],    
          'max_depth': [7,10,25],
          'max_features': [100,200],
          'min_samples_split' : [0.01,0.05]
         }

grid = GridSearchCV(estimator = clf,
                          param_grid = params,
#                           scoring = make_scorer(roc_auc_score),
                          cv = 8 
)

inicio = time.time()
grid.fit(X_train_3, y_train)
fim = time.time()

print(fim - inicio)

print(grid.best_score_)
classif_final_RF = grid.best_estimator_
print(grid.best_estimator_)
print(grid.best_params_)

In [ ]:
pred_test = classif_final_RF.predict(X_test_3)
prob_test = classif_final_RF.predict_proba(X_test_3)
pred_train = classif_final_RF.predict(X_test_3)
prob_train = classif_final_RF.predict_proba(X_test_3)

y_true = np.array(y_test.copy())
y_pred = np.array(pred_test.copy())

from sklearn.metrics import confusion_matrix
matriz_confusao = pd.DataFrame(confusion_matrix(y_true,y_pred)) 
print(matriz_confusao)

In [ ]:
y_true_treino = np.array(y_test.copy())
y_pred_treino = np.array(pred_train.copy())

y_test_score = prob_test.copy()
y_test_score = pd.DataFrame(y_test_score)
y_test_score = y_test_score.iloc[:,1]

y_train_score = prob_train.copy()
y_train_score = pd.DataFrame(y_train_score)
y_train_score = y_train_score.iloc[:,1]

### Calculo do *KS*

In [ ]:
KS  = matriz_confusao.iloc[0,0]/(matriz_confusao.iloc[0,1] + matriz_confusao.iloc[0,0]) -  matriz_confusao.iloc[1,0]/(matriz_confusao.iloc[1,0] + matriz_confusao.iloc[1,1])
KS

### ------------------------------------------------------------------------------------------------------------------------------------------------------------

## Gradient Boosting

In [ ]:
# GB
clf2 = GradientBoostingClassifier(random_state = 665 )
params2 = {'n_estimators' : [100,300],    
           'max_depth': [2,3,4],
           'learning_rate': [0.1,0.05],
           'min_samples_split' : [0.02]
          }
grid = GridSearchCV(estimator = clf2,
                          param_grid = params2,
                          scoring = make_scorer(roc_auc_score),
                          cv = 10
)
grid.fit(X_train_3, y_train)
print(grid.score(X_train_3, y_train))
classif_final_GB = grid.best_estimator_
print(grid.best_estimator_)
# features_imp = pd.DataFrame(X_train_3.columns)
# features_imp['importance'] = classif_final.feature_importances_
# features_imp=features_imp.sort_values('importance', ascending=False)

# features_imp

In [ ]:
pred_test = classif_final_GB.predict(X_test_3)
prob_test = classif_final_GB.predict_proba(X_test_3)
pred_train = classif_final_GB.predict(X_test_3)
prob_train = classif_final_GB.predict_proba(X_test_3)

In [ ]:
y_true = np.array(y_test.copy())
y_pred = np.array(pred_test.copy())

In [ ]:
len(y_true)

In [ ]:
len(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
matriz_confusao = pd.DataFrame(confusion_matrix(y_true,y_pred)) 
matriz_confusao

In [ ]:
# print("\n")
# print(y_true.sum())
# print("\n")
# print(y_pred.sum())
# print("\n")

In [ ]:
y_true_treino = np.array(y_test.copy())
y_pred_treino = np.array(pred_train.copy())

y_test_score = prob_test.copy()
y_test_score = pd.DataFrame(y_test_score)
y_test_score = y_test_score.iloc[:,1]

y_train_score = prob_train.copy()
y_train_score = pd.DataFrame(y_train_score)
y_train_score = y_train_score.iloc[:,1]

### Calculo do *KS*

In [ ]:
KS  = matriz_confusao.iloc[0,0]/(matriz_confusao.iloc[0,1] + matriz_confusao.iloc[0,0]) -  matriz_confusao.iloc[1,0]/(matriz_confusao.iloc[1,0] + matriz_confusao.iloc[1,1])
KS

### Tabela como o calculo de varias metricas 

In [ ]:
data = {'Arvore': [roc_auc_score(y_true,y_pred),f1_score(y_true,y_pred, pos_label = 1),precision_score(y_true,y_pred),recall_score(y_true,y_pred),accuracy_score(y_true,y_pred),KS]}
data = pd.DataFrame(data)
data.index = ['ROC', 'F1','Precision','Recall','Acuracia','KS']
data

### Plot da curva ROC 

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_true, y_test_score ,pos_label=1)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(false_positive_rate, true_positive_rate, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show() 

### ------------------------------------------------------------------------------------------------------------------------------------------------------------

## Arvores de Decisão 

In [ ]:
#ExtraTrees 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier

ExtC = ExtraTreesClassifier()

## Search grid for optimal parameters
ex_param_grid = {"max_depth": [20,25],
              "max_features": [130,150,180],
              "min_samples_split": [0.05,0.3],
              "min_samples_leaf": [10, 15, 5],
              "bootstrap": [True, False],
              "n_estimators" :[250,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=10, scoring="accuracy", n_jobs= -1, verbose = 1)

gsExtC.fit(X_train_3,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [ ]:
gsExtC.best_params_

In [ ]:
pred_test = ExtC_best.predict(X_test_3)
prob_test = ExtC_best.predict_proba(X_test_3)
pred_train = ExtC_best.predict(X_test_3)
prob_train = ExtC_best.predict_proba(X_test_3)

y_true = np.array(y_test.copy())
y_pred = np.array(pred_test.copy())

y_true_treino = np.array(y_test.copy())
y_pred_treino = np.array(pred_train.copy())

y_test_score = prob_test.copy()
y_test_score = pd.DataFrame(y_test_score)
y_test_score = y_test_score.iloc[:,1]

y_train_score = prob_train.copy()
y_train_score = pd.DataFrame(y_train_score)
y_train_score = y_train_score.iloc[:,1]

In [ ]:
data = {'Tree': [roc_auc_score(y_true,y_pred),f1_score(y_true,y_pred, pos_label = 1),precision_score(y_true,y_pred),recall_score(y_true,y_pred),accuracy_score(y_true,y_pred),KS]}
data = pd.DataFrame(data)
data.index = ['ROC', 'F1','Precision','Recall','Acuracia','KS']
data

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred = gsExtC.predict(X_test_3)
print(confusion_matrix(y_test, y_pred))

### ------------------------------------------------------------------------------------------------------------------------------------------------------------

## MLP - MultiLayer 

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(max_iter=100,random_state=123,verbose=False)

params2 = {
    'hidden_layer_sizes': [(50,100,50)],
#        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['relu'],
#        'activation': ['relu', 'identity', 'tanh', 'logistic'],
    'solver': ['adam'],
#        'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.05],
#        'alpha': [0.0001, 0.05],]
    'learning_rate': ['constant'],
#        'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init':[0.1],
#        'learning_rate_init':[0.001,0.01,0.1,0.2,0.3],

    
}

grid = GridSearchCV(estimator = clf,
                          param_grid = params2,
#                           scoring = make_scorer(roc_auc_score),
                          cv = 10
)

grid.fit(X_train_3, y_train)

print(grid.best_estimator_)

### Melhor estimador 

In [ ]:
classif_final_MLP = grid.best_estimator_
classif_final_MLP

In [ ]:
pred_test = classif_final_MLP.predict(X_test_3)
prob_test = classif_final_MLP.predict_proba(X_test_3)
pred_train = classif_final_MLP.predict(X_test_3)
prob_train = classif_final_MLP.predict_proba(X_test_3)


y_true = np.array(y_test.copy())
y_pred = np.array(pred_test.copy())


In [ ]:
y_true_treino = np.array(y_test.copy())
y_pred_treino = np.array(pred_train.copy())

y_test_score = prob_test.copy()
y_test_score = pd.DataFrame(y_test_score)
y_test_score = y_test_score.iloc[:,1]

y_train_score = prob_train.copy()
y_train_score = pd.DataFrame(y_train_score)
y_train_score = y_train_score.iloc[:,1]

In [ ]:
KS  = matriz_confusao.iloc[0,0]/(matriz_confusao.iloc[0,1] + matriz_confusao.iloc[0,0]) -  matriz_confusao.iloc[1,0]/(matriz_confusao.iloc[1,0] + matriz_confusao.iloc[1,1])
KS

### Tabela com as metricas 

In [ ]:
data = {'MLP': [roc_auc_score(y_true,y_pred),f1_score(y_true,y_pred, pos_label = 1),precision_score(y_true,y_pred),recall_score(y_true,y_pred),accuracy_score(y_true,y_pred),KS]}
data = pd.DataFrame(data)
data.index = ['ROC', 'F1','Precision','Recall','Acuracia','KS']
data

### Matriz de Confusão 

In [ ]:
from sklearn.metrics import confusion_matrix
matriz_confusao = pd.DataFrame(confusion_matrix(y_true,y_pred)) 
matriz_confusao

## -------------------------------------------------------------------------------------------------------------------------------

## Logistic Regression

In [ ]:

# Training Logistic regression model
clf5 = LogisticRegression()

params4 = {'C'  :[0.8, 0.5,0.3]}

          
grid = GridSearchCV(estimator = clf5,
                          param_grid = params4,
                          #scoring = 
                          cv = 5
)

grid.fit(X_train_3, y_train)

print(grid.score(X_train_3, y_train))
classif_final_Log = grid.best_estimator_
print(grid.best_estimator_)

In [ ]:
pred_test = classif_final_Log.predict(X_test_3)
prob_test = classif_final_Log.predict_proba(X_test_3)
pred_train = classif_final_Log.predict(X_test_3)
prob_train = classif_final_Log.predict_proba(X_test_3)

y_true = np.array(y_test.copy())
y_pred = np.array(pred_test.copy())

In [ ]:
y_true_treino = np.array(y_test.copy())
y_pred_treino = np.array(pred_train.copy())

y_test_score = prob_test.copy()
y_test_score = pd.DataFrame(y_test_score)
y_test_score = y_test_score.iloc[:,1]

y_train_score = prob_train.copy()
y_train_score = pd.DataFrame(y_train_score)
y_train_score = y_train_score.iloc[:,1]

In [ ]:
KS  = matriz_confusao.iloc[0,0]/(matriz_confusao.iloc[0,1] + matriz_confusao.iloc[0,0]) -  matriz_confusao.iloc[1,0]/(matriz_confusao.iloc[1,0] + matriz_confusao.iloc[1,1])
KS

In [ ]:
data = {'Logistic': [roc_auc_score(y_true,y_pred),f1_score(y_true,y_pred, pos_label = 1),precision_score(y_true,y_pred),recall_score(y_true,y_pred),accuracy_score(y_true,y_pred),KS]}
data = pd.DataFrame(data)
data.index = ['ROC', 'F1','Precision','Recall','Acuracia','KS']
data

In [ ]:
from sklearn.metrics import confusion_matrix
matriz_confusao = pd.DataFrame(confusion_matrix(y_true,y_pred)) 
matriz_confusao

## -----------------------------------------------------------------------------------------------------------------------------

In [ ]:
from sklearn.svm import SVC
#print(grid.score(X_train_3, y_train))

svc= SVC(kernel='poly',degree=3,probability=True)
classif_final_SVM = svc.fit(X_train_3,y_train)


In [ ]:
pred_test = classif_final_SVM.predict(X_test_3)
prob_test = classif_final_SVM.predict_proba(X_test_3)
pred_train = classif_final_SVM.predict(X_test_3)
prob_train = classif_final_SVM.predict_proba(X_test_3)

y_true = np.array(y_test.copy())
y_pred = np.array(pred_test.copy())

In [ ]:
y_true_treino = np.array(y_test.copy())
y_pred_treino = np.array(pred_train.copy())

y_test_score = prob_test.copy()
y_test_score = pd.DataFrame(y_test_score)
y_test_score = y_test_score.iloc[:,1]

y_train_score = prob_train.copy()
y_train_score = pd.DataFrame(y_train_score)
y_train_score = y_train_score.iloc[:,1]

In [ ]:
KS  = matriz_confusao.iloc[0,0]/(matriz_confusao.iloc[0,1] + matriz_confusao.iloc[0,0]) -  matriz_confusao.iloc[1,0]/(matriz_confusao.iloc[1,0] + matriz_confusao.iloc[1,1])
KS

In [ ]:
data = {'SVM': [roc_auc_score(y_true,y_pred),f1_score(y_true,y_pred, pos_label = 1),precision_score(y_true,y_pred),recall_score(y_true,y_pred),accuracy_score(y_true,y_pred),KS]}
data = pd.DataFrame(data)
data.index = ['ROC', 'F1','Precision','Recall','Acuracia','KS']
data

In [ ]:
from sklearn.metrics import confusion_matrix
matriz_confusao = pd.DataFrame(confusion_matrix(y_true,y_pred)) 
matriz_confusao

In [ ]:
# Arvore
clf3 =  DecisionTreeClassifier(random_state = 665)
params3 = {'min_samples_split' : [85,100,150], 
           'max_depth': [2,3,4,5,6],
           'min_samples_leaf':[80,100,150]
          }
grid = GridSearchCV(estimator = clf3,
                          param_grid = params3,
#                           scoring = 
                          cv = 10
)

grid.fit(X_train_3, y_train)

print(grid.score(X_train_3,y_train))
classif_final_Tree = grid.best_estimator_
print(grid.best_estimator_)



In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris()
clf = tree.DecisionTreeClassifier()


clf = clf.fit(iris.data, iris.target)

In [ ]:
tree.plot_tree(clf.fit(X_train_3,y_train)) 